In [ ]:
import json

with open('IR_data_news_12k.json') as f:
    data = json.load(f)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data['0'].keys()

dict_keys(['title', 'content', 'tags', 'date', 'url', 'category'])

In [ ]:
from elasticsearch import Elasticsearch, helpers
import os, uuid
# import Python's JSON library for its loads() method
import json

# import time for its sleep method
from time import sleep

# import the datetime libraries datetime.now() method
from datetime import datetime
from tqdm import tqdm 

In [ ]:
es = Elasticsearch("http://localhost:9200")

In [ ]:
texts = [
    "مسابقات ایران و روسیه",
    "ظریف در وین",
    "اقدامات وزارت بهداشت"
]

In [ ]:
def get_query(text):
    body ={
    "query":{  
        "match" : {
            "content" : text

            }
        }
    }
    
    return body



# Create tf-idf Index

In [ ]:
es.indices.delete(index="tfidf_index")
es.indices.create(index="tfidf_index"                    
                 
                 )

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'tfidf_index'})

In [ ]:
for i in tqdm(range(len(data))):
    es.index(index='tfidf_index',
        id=i, body=data[str(i)])
    i = i + 1

100%|█████████████████████████████████████| 12202/12202 [20:23<00:00,  9.97it/s]


In [ ]:
es.indices.close(index="tfidf_index")
es.indices.put_settings(index="tfidf_index", 
                        settings={
                            "similarity": {
                                  "default": {
                                    "type": "scripted",
                                    "script": {
                                      "source": "double tf = Math.sqrt(doc.freq); double idf = Math.log((field.docCount+1.0)/(term.docFreq+1.0)) + 1.0; double norm = 1/Math.sqrt(doc.length); return query.boost * tf * idf * norm;"
                                    }
                                  }
                            }
                        }
                       )
es.indices.open(index="tfidf_index")

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True})

In [ ]:
all_res_tfidf = []
for t in texts:
    res_tfidf = es.search(index="tfidf_index", body=get_query(t), explain=True)
    all_res_tfidf.append(dict(res_tfidf))

In [ ]:
for a in all_res_tfidf:
    print(a, end="_______________\n")

In [ ]:
for res in all_res_tfidf:
    for doc in res['hits']['hits']:
        print(doc['_source']['title'])
    print("----------------------------")

ترکش اختلافات به کشتی برخورد کرد/انصراف روسیه از مسابقات قهرمانی اروپا
فیفا و یوفا روسیه را از تمام مسابقات فوتبال کنار گذاشتند
واکنش حسینی به عصبانیت غربگرایان / روابط تهران و مسکو بر اساس احترام و منافع متقابل است
سفر رئیسی به روسیه پیامدهای مهمی برای ثبات منطقه و شکوفایی اقتصادی دارد
پاراتیراندازی با کمان قهرمانی جهان| قهرمانی و مدال طلای کامپوند مردان /بانوان به مدال برنز رسیدند
جام ملت های فوتسال اروپا|  اسپانیا باز هم در مقابل پرتغال ناکام ماند/صعود سخت روس‌ها به فینال
رئیس جمهور: مقدمات نهایی شدن سند جامع همکاری‌های ایران و روسیه فراهم شده است
جام ملت های فوتسال اروپا | پرتغال قهرمان شد
وزارت ورزش روسیه دست به کار شد/آغاز رایزنی برای کاهش محرومیت‌ها
اعتراض روسی‌ها به تصمیمات فیفا و کمیته ملی المپیک
----------------------------
پاسخ خواندنی شریعتمداری به ادعای یک اصلاح‌طلب
مذاکره زیر سایه تزویر/ وقتی ظریف پیگیر انجام آزمایش موشکی بود
عضو دفتر جلیلی، نامه ۲۰۰ صفحه‌ای منتسب به وی را «خیالی» خواند
توضیحات قالیباف درباره مذاکرات وین در جلسه غیرعلنی مجلس
بهادری‌جهرمی: سیاست‌خارجی نتیج

# Create BM25 Index

In [ ]:
es.indices.delete(index="my_index")
es.indices.create(index="my_index"                    
                 
                 )

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'})

In [ ]:
for i in tqdm(range(len(data))):
    es.index(index='my_index',
        id=i, body=data[str(i)])
    i = i + 1

100%|█████████████████████████████████████| 12202/12202 [18:33<00:00, 10.96it/s]


In [ ]:
es.indices.close(index="my_index")
es.indices.put_settings(index="my_index", 
                        settings={
                            "similarity": {
                                  "default": {
                                    "type": "BM25"
                                  }
                            }
                        }
                       )
es.indices.open(index="my_index")

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True})

In [ ]:
all_res_bm = []
for t in texts:
    res_bm = es.search(index="my_index", body=get_query(t), explain=True)
    all_res_bm.append(dict(res_bm))

In [ ]:
for a in all_res_bm:
    print(a, end="_______________\n")

In [ ]:
for res in all_res_bm:
    for doc in res['hits']['hits']:
        print(doc['_source']['title'])
    print("----------------------------")

حریفان والیبال ایران در لیگ ملت‌ها مشخص شدند/ هفته نفسگیر شاگردان عطایی در صوفیه
برنامه مسابقات تیم ملی والیبال در لیگ ملت‌ها/ تقابل با دو حریف آسیایی در کمتر از 16 ساعت
داورزنی تاریخ بازی‌های آسیایی را جا به‌جا می‌کند
اتحادیه جهانی اعلام کرد؛ کشتی فرنگی ایران در بین بهترین‌های سال ۲۰۲۱
رقیب حسن یزدانی در روسیه محک می‌خورد/کدام مدال‌آوران آمریکا به جام یاریگین می‌روند؟
ترکش اختلافات به کشتی برخورد کرد/انصراف روسیه از مسابقات قهرمانی اروپا
سرلشکر باقری: قراردادهای خرید تسلیحاتی در سفر به روسیه پیگیری می‌شود/ تبادل نظر ایران و روسیه درباره افغانستان
پاراتیراندازی با کمان قهرمانی جهان| قهرمانی و مدال طلای کامپوند مردان /بانوان به مدال برنز رسیدند
قهرمان یخ‌نوردی دنیا: به جنگ حریفانی می‌رویم که تا دندان مسلح‌اند/ فقط 10 روز تمرین داشتیم
فیفا و یوفا روسیه را از تمام مسابقات فوتبال کنار گذاشتند
----------------------------
وقتی اصلاح‌طلبان نمی‌فهمند چه چیزی را امضا کرده‌اند/ تغییر زمین بازی برای پاس کردن چک برگشتی برجام
پاسخ خواندنی شریعتمداری به ادعای یک اصلاح‌طلب
مذاکره زیر سایه تزویر/ وقت

# Create LMDirichlet Index

In [ ]:
es.indices.delete(index="LMDirichlet_index")
es.indices.create(index="LMDirichlet_index"                    
                 
                 )

In [ ]:
for i in tqdm(range(len(data))):
    es.index(index='LMDirichlet_index',
        id=i, body=data[str(i)])
    i = i + 1

In [ ]:
es.indices.close(index="LMDirichlet_index")
es.indices.put_settings(index="LMDirichlet_index", 
                        settings={
                            "similarity": {
                                  "default": {
                                    "type": "LMDirichlet"
                                  }
                            }
                        }
                       )
es.indices.open(index="LMDirichlet_index")

In [ ]:
body2 ={
    "query":{         
        "match" : {
#             "content" : "مشکل کم آبی و محیط زیست در کشور ایران"
            "content" : "ایران در خاورمیانه"

        }
            
    }
}

res_LMDirichlet = es.search(index="LMDirichlet_index", body=body2, explain=True)
res_LMDirichlet = dict(res_LMDirichlet)

In [ ]:
res_LMDirichlet

In [ ]:
for doc in res_LMDirichlet['hits']['hits']:
    print(doc['_source']['title'])